In [191]:
import os

import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing as preprocessing
# optimizer 문제: https://developer.apple.com/forums/thread/721735
from tensorflow.keras.optimizers.legacy import Adam

# CNN 기본 설명
합성곱(convolution): 데이터 위를 필터(윈도우, 커널, 마스크 등 다양하게 불림)가 지정된 스트라이드씩
슬라이딩하며 필터가 나타내는 특징을 추출한다. 마지막 위치까지 슬라이딩을 마치면 찾고자
했던 특징이 데이터 전체에서 나타난 '특징맵(feature map)'이 출력되는 연산이다.

풀링(pooling): 합성곱과 마찬가지로 윈도우가 있으며 데이터 위를 슬라이딩한단 점은 동일하다. 차이점은
슬라이딩마다 윈도우 범위 내에서 가장 특징이 뚜렷한 지점의 값을 찾거나, 윈도우 범위 내 특징의 평균값을
낸다는 점이다. 전자를 최대 풀링(max pooling), 후자를 평균 풀링(average pooling) 연산이라고 부른다.

합성곱으로 피처맵 추출 -> 피처맵에서 최대 풀링 연산으로 가장 뚜렷한 특징값을 집약 순서로 레이어를 구성한다.

# 데이터와 모델 설명
데이터(인공): [Chatbot_data_for_Korean v1.0](https://github.com/songys/Chatbot_data)
필드: Q(사용자 입력), A(가상의 챗봇 반응), label(감정, 0: 일상적, 1: 이별, 2: 사랑)

In [201]:
def features_and_labels(csv_file):
    df = pd.read_csv(csv_file)
    features, labels = df["Q"].tolist(), df["label"].tolist()
    return features, labels


def dataset_and_metadata(features, labels, batch_size=20):
    """데이터셋과 그에 관해 설명하는 메타데이터.

    :param features: `features_and_labels`의 첫번째 리턴.
    :param labels: `features_and_labels`의 두번째 리턴.
    :return: 데이터셋과 그에 관한 메타데이터. 메타데이터는 프로그램 중간중간 필요하기에 뽑았다.
    """
    # `text_to_word_sequence` 메소드로 텍스트를 토큰 단위로 나눈 단어 시퀀스 생성.
    tokens = [preprocessing.text.text_to_word_sequence(document)
              for document in features]

    tokenizer = preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(tokens)
    # 각 문서 내 토큰을 숫자로 치환. `texts_to_sequences`는 `fit_on_texts` 메서드에 인자로 준
    # 토큰만 인식할 수 있다. 빈도가 높은 순으로 (num_words - 1)개의 단어만 고려한다.
    int_sequences = tokenizer.texts_to_sequences(tokens)
    index_word = tokenizer.index_word  # 토큰에 어떤 번호를 부여했는지 매핑 객체.

    # CNN 모델은 고정 크기 벡터를 요구한다. 고정 크기 설정 시 벡터의 길이보다 짧게 잡으면 정보 손실이
    # 일어나므로 문서들에서 가장 긴 벡터의 길이를 알아야 한다.
    lens_sequences = list(map(lambda vector: len(vector), int_sequences))
    fixed_len_sequence = 15  # 테스트셋까지 고려하면 15가 최소 길이. max(lens_sequences)
    padded_int_sequences = preprocessing.sequence.pad_sequences(
        int_sequences,
        maxlen=fixed_len_sequence,
        padding="post")  # 벡터 뒤에 패딩을 붙인다. 기본은 'pre'.

    # 인자로 받은 텐서들을 잘라내(slice) 데이터셋을 만든다. 여기선 파이썬의 `zip`함수를 생각하면 된다.
    # `zip(features, labels)`은 각 순회마다 `(feature[i], labels[i])를 출력한다.
    # 이와 마찬가지로 `from_tensor_slices`도 `(feat, lbl)`로 묶인 TensorSliceDataset을 리턴.
    dataset = tf.data.Dataset.from_tensor_slices((padded_int_sequences, labels))
    dataset.shuffle(len(padded_int_sequences))
    dataset = dataset.batch(batch_size)

    metadata = {
        "vocab_size": len(index_word) + 1,  # 데이터셋 내 모든 구별되는 단어 개수.
        "fixed_len_sequence": fixed_len_sequence}

    return dataset, metadata

In [202]:
# 데이터셋 생성
data_dir = "data/corpus/chatbot_data"
train_file = os.path.join(data_dir, "trainset.csv")
test_file = os.path.join(data_dir, "testset.csv")

train_features, train_labels = features_and_labels(train_file)
test_features, test_labels = features_and_labels(test_file)

batch_size = 20
trainset, train_metadata = dataset_and_metadata(
    train_features,
    train_labels,
    batch_size)
testset, test_metadata = dataset_and_metadata(
    test_features,
    test_labels,
    batch_size)

In [203]:
print(trainset)

<BatchDataset element_spec=(TensorSpec(shape=(None, 15), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [204]:
# 모델 하이퍼파라미터 설정
dropout_prob = 0.5
emb_size = 128  # 임베딩 벡터 크기.
epoch = 5
eval_split = 0.7
# `texts_to_sequences`는 top(num_words - 1)만 고려하는 특성이 있어서 1을 더해야 한다.
logit_space_size = len(set(train_labels))  # 분류할 감정은 0, 1, 2 세 가지다.

In [205]:
input_layer = tf.keras.layers.Input(shape=(train_metadata["fixed_len_sequence"],))
embedding_layer = tf.keras.layers.Embedding(
    input_dim=train_metadata["vocab_size"],
    output_dim=emb_size,
    input_length=train_metadata["fixed_len_sequence"])(input_layer)
dropout_emb_layer = tf.keras.layers.Dropout(rate=dropout_prob)(embedding_layer)

In [206]:
conv_layer1 = tf.keras.layers.Conv1D(
    filters=emb_size,  # Integer, the dimensionality of the output space
    kernel_size=3,  # 길이 3인 벡터로 n-gram의 n과 같은 역할을 수행.
    padding="valid",  # 패딩 없음.
    activation=tf.nn.relu,
    name="conv1")(dropout_emb_layer)
pooling_layer1 = tf.keras.layers.GlobalMaxPool1D()(conv_layer1)

In [207]:
conv_layer2 = tf.keras.layers.Conv1D(
    filters=emb_size,
    kernel_size=4,
    padding="valid",
    activation=tf.nn.relu,
    name="conv2")(dropout_emb_layer)
pooling_layer2 = tf.keras.layers.GlobalMaxPool1D()(conv_layer2)

In [208]:
conv_layer3 = tf.keras.layers.Conv1D(
    filters=emb_size,
    kernel_size=5,  # n-gram 크기를 키우면서 넓은 문맥의 의미도 추출한다.
    padding="valid",
    activation=tf.nn.relu,
    name="conv3")(dropout_emb_layer)
pooling_layer3 = tf.keras.layers.GlobalMaxPool1D()(conv_layer3)

In [209]:
pooling_layer_sequence = [pooling_layer1, pooling_layer2, pooling_layer3]
concat = tf.keras.layers.concatenate(pooling_layer_sequence)

In [210]:
dense_layer = tf.keras.layers.Dense(
    units=emb_size,
    activation=tf.nn.relu)(concat)
dropout_dense_layer = tf.keras.layers.Dropout(rate=dropout_prob)(dense_layer)
logit_layer = tf.keras.layers.Dense(
    units=logit_space_size,
    name="logits")(dropout_dense_layer)
prediction_layer = tf.keras.layers.Dense(
    units=logit_space_size,
    activation=tf.nn.softmax)(logit_layer)  # 엔트로피 함수로 학습하기 위해선 출력이 확률이어야 한다.

In [211]:
model = tf.keras.Model(inputs=input_layer, outputs=prediction_layer)
model.compile(
    optimizer=Adam(),  # "adam" 대신. tensorflow-macos 2.11에서 호환성 문제가 있다.
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [212]:
model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, 15, 128)      1440000     ['input_9[0][0]']                
                                                                                                  
 dropout_19 (Dropout)           (None, 15, 128)      0           ['embedding_8[0][0]']            
                                                                                                  
 conv1 (Conv1D)                 (None, 13, 128)      49280       ['dropout_19[0][0]']             
                                                                                           

In [213]:
model.fit(trainset, epochs=epoch, verbose=1)

Epoch 1/5


2023-01-17 04:24:14.734924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


473/473 [==============================] - 8s 16ms/step - loss: 0.3888 - accuracy: 0.9401
Epoch 2/5
473/473 [==============================] - 7s 15ms/step - loss: 0.5998 - accuracy: 0.8383
Epoch 3/5
473/473 [==============================] - 7s 15ms/step - loss: 0.4035 - accuracy: 0.8695
Epoch 4/5
473/473 [==============================] - 7s 15ms/step - loss: 0.3499 - accuracy: 0.9076
Epoch 5/5
473/473 [==============================] - 7s 15ms/step - loss: 0.4163 - accuracy: 0.8725


In [214]:
loss, accuracy = model.evaluate(testset, verbose=1)
print(f"Accuracy: {accuracy}, loss: {loss}")

 14/119 [==>...........................] - ETA: 0s - loss: 0.3146 - accuracy: 1.0000

2023-01-17 04:24:51.147691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


119/119 [==============================] - 1s 7ms/step - loss: 0.3334 - accuracy: 0.9992
Accuracy: 0.9991543889045715, loss: 0.33344805240631104


In [217]:
model_save_path = "models/intent_classifier"
model.save(model_save_path)

INFO:tensorflow:Assets written to: models/intent_classifier/assets


INFO:tensorflow:Assets written to: models/intent_classifier/assets
